# 청와대 청원 크롤링 코드 

## 작성자: 박하람
## 작성일자: 2020년 8월 10일 

분야별 청원 전체 카테고리 선택 후, 추천 순 정렬하여 모든 청원 가져오기

In [2]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [3]:
from urllib.request import urlopen
import requests 
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import time
from random import randint
import re

In [34]:
def get_peti_df(start, end): 
    
    category = []
    titles = []
    address = []
    end_date = []
    consent_num = []
    
    for n in tqdm(range(start, end+1)): 
        print(n)
        session = requests.Session()
        url = f'https://www1.president.go.kr/petitions/?c=0&only=2&page={n}&order=2' 
        headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
                  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
        MAX_SLEEP_TIME = 3
        rand_value = randint(1, MAX_SLEEP_TIME)
        time.sleep(rand_value)
        html = session.get(url, headers=headers).content
        soup = BeautifulSoup(html, "html.parser")
        
        all_list = soup.find('div', class_="ct_list1")
        contents = all_list.find_all('div', class_='bl_body')
         
        for i in range(7):
            peti_list = contents[0].find_all('div', class_='bl_wrap')[i]
            
            # 청원 카테고리
            category.append(peti_list.find('div', class_='bl_category').get_text()) 
            # 청원 제목 
            titles_raw = peti_list.find('div', class_='bl_subject').get_text()
            titles_split = re.split('\r|\t|\n|\xa0|제목', titles_raw)
            titles.append(''.join(titles_split))
            # 청원 주소 
            add_url = peti_list.find('a')['href']
            address.append('https://www1.president.go.kr'+add_url)
            # 청원 만료일 
            end_date.append(peti_list.find('div', class_='bl_date').get_text())
            # 청원 동의 수 
            consent_num_raw = peti_list.find('div', class_='bl_agree').get_text()
            consent_split = re.split('\r|\t|\n|\xa0', consent_num_raw)
            consent_num.append(''.join(consent_split).split()[1][:-1])
        
    petition_dict = {'카테고리':category, '제목':titles, '청원종료일': end_date, 
                 '참여인원': consent_num,'URL': address}
    df = pd.DataFrame(petition_dict)
    return df 
        

In [51]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www1.president.go.kr/petitions/?c=0&only=2&page=3&order=2' 
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")
        
contents = all_list.find_all('div', class_='bl_body')

In [52]:
contents

[<div class="bl_body">
 <ul class="petition_list">
 </ul>
 </div>]

In [ ]:
#cont_view > div.cs_area > div > div > div.ct_petitions > div.ct_petitions_area.ct_txt_left > div.ct_list1 > div > div.b_list.category.b_list2 > div.bl_body

In [37]:
%time peti_df = get_peti_df(1,2)

  0%|          | 0/2 [00:00<?, ?it/s]

1


  0%|          | 0/2 [00:01<?, ?it/s]


IndexError: list index out of range

In [29]:
%time peti_df = get_peti_df(1,300)

  0%|          | 0/300 [00:00<?, ?it/s]

1


  0%|          | 1/300 [00:01<06:17,  1.26s/it]

2


  1%|          | 2/300 [00:02<06:18,  1.27s/it]

3


  1%|          | 2/300 [00:03<09:13,  1.86s/it]


KeyboardInterrupt: 

In [30]:
len(peti_df)

140

In [31]:
peti_df

,카테고리,제목,청원종료일,참여인원,URL
0,일자리,공기업 비정규직의 정규화 그만해주십오.,20.07.23,"352,266",https://www1.president.go.kr/petitions/589941
1,인권/성평등,'저의 딸이 강간 당하는 것을 목격하여..' 그 충격과 고통으로 딸이 평생 남을...,20.07.11,"286,148",https://www1.president.go.kr/petitions/589634
2,안전/환경,응급환자가 있는 구급차를 막아세운 택시 기사를 처벌해 주세요.,20.08.02,"735,972",https://www1.president.go.kr/petitions/590341
3,인권/성평등,강영수 서울고법 수석부장판사의 대법관 후보 자격 박탈을 청원합니다.,20.08.05,"529,144",https://www1.president.go.kr/petitions/590416
4,행정,정부의 교회 정규 예배 이외 행사 금지를 취소해주세요.,20.08.07,"427,470",https://www1.president.go.kr/petitions/590504
...,...,...,...,...,...
135,안전/환경,응급환자가 있는 구급차를 막아세운 택시 기사를 처벌해 주세요.,20.08.02,"735,972",https://www1.president.go.kr/petitions/590341
136,인권/성평등,강영수 서울고법 수석부장판사의 대법관 후보 자격 박탈을 청원합니다.,20.08.05,"529,144",https://www1.president.go.kr/petitions/590416
137,행정,정부의 교회 정규 예배 이외 행사 금지를 취소해주세요.,20.08.07,"427,470",https://www1.president.go.kr/petitions/590504
138,인권/성평등,"박원순씨 장례를 5일장, 서울특별시장(葬)으로 하는 것 반대합니다.",20.08.09,"596,410",https://www1.president.go.kr/petitions/590550


In [32]:
peti_df.sort_values('참여인원', ascending=False).tail()

,카테고리,제목,청원종료일,참여인원,URL
69,인권/성평등,강서구 데이트폭력 살인미수사건 강력한 처벌을 촉구합니다,20.07.31,"212,867",https://www1.president.go.kr/petitions/590258
97,인권/성평등,강서구 데이트폭력 살인미수사건 강력한 처벌을 촉구합니다,20.07.31,"212,867",https://www1.president.go.kr/petitions/590258
76,인권/성평등,강서구 데이트폭력 살인미수사건 강력한 처벌을 촉구합니다,20.07.31,"212,867",https://www1.president.go.kr/petitions/590258
83,인권/성평등,강서구 데이트폭력 살인미수사건 강력한 처벌을 촉구합니다,20.07.31,"212,867",https://www1.president.go.kr/petitions/590258
139,인권/성평등,강서구 데이트폭력 살인미수사건 강력한 처벌을 촉구합니다,20.07.31,"212,867",https://www1.president.go.kr/petitions/590258


In [33]:
peti_df.to_csv('blue-house-petition-best.csv', sep=',', encoding='utf-8')

In [ ]:
import pandas as pd 

petition = pd.read_csv('blue-house-petition-best.csv', thousands=',', encoding='utf-8', index_col=0)
petition.head()

In [ ]:
import re 
import time
from tqdm import tqdm
from urllib.request import urlopen
from bs4 import BeautifulSoup

content = []

for n in tqdm(petition.index): 
    session = requests.Session()
    url = petition['URL'][n]
    headers = {"User-Agent": 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36',
                  "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}
    MAX_SLEEP_TIME = 3
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)
    html = session.get(url, headers=headers).content
    soup = BeautifulSoup(html, "html.parser") 
    
    # get petition content 
    content_raw = soup.find_all('div', class_='View_write')
    try: 
        if len(content_raw) > 1:
            content_raw_text = content_raw[1].get_text()
        else: 
            content_raw_text = content_raw[0].get_text()   
    except IndexError: 
        print('IndexError:', petition['URL'][n])
    
    content_split = re.split('\r|\t|\n|\xa0', content_raw_text)
    content.append(''.join(content_split))

In [ ]:
petition['청원내용'] = content

In [ ]:
petition.to_csv('final-blue-house-petition-best.csv', sep=',', encoding='utf-8')